In [310]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

In [327]:
#parameters

sample_rate=10
no_of_lines =50
line_tolarance=1.5

In [320]:
def process_data(file, sample_rate = sample_rate):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    point_array_sampled = []
    for i in range(0,len(coord),sample_rate):
        point_array_sampled.append(coord[i])
        
    point_array_sampled = np.array(point_array_sampled)
        
    return np.array( point_array_sampled)

In [321]:
def plot_graph(point_data,outline_points=None, lines=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create cubic bounding box to simulate equal aspect ratio
    X = point_data[:,0]
    Y = point_data[:,1]
    Z = point_data[:,2]

    ax.scatter(point_data[:,0], point_data[:,1], point_data[:,2], s=1)

    #bounding box to show foot in scale
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())

    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')
        
    if (outline_points is not None):
        flatten = outline_points.reshape(no_of_lines*2,3)
        ax.scatter(flatten[:,0], flatten[:,1], flatten[:,2], s=500/no_of_lines, color='red')

    plt.grid()
    plt.show()

In [322]:
def project_points(point_data, axis=1, height_limit=100000): #axis values x=0/y=1/z=2 implement limit
    if(axis==0):
        point_array[:,0]=0
    if(axis==1):
        point_array[:,1]=0
    if(axis==2):
        point_array[:,2]=0
        
    return point_array

In [323]:
def get_outline(point_data, no_of_lines = no_of_lines, line_tolarance = line_tolarance):
    
    z_min = min(point_data[:,2])
    z_max =max(point_data[:,2])
    
    step = (z_max-z_min)/(no_of_lines+1)
    
    lines = np.arange(z_min, z_max, step)[1:]
    
    outline_points=[]
    
    for line in lines:
        line_points=[]
        candidates = point_data[np.where(np.logical_and(point_data[:,2]>line, point_data[:,2]<line+line_tolarance))]
        line_points.append(candidates[np.where(candidates[:,0]==min(candidates[:,0]))][0])
        line_points.append(candidates[np.where(candidates[:,0]==max(candidates[:,0]))][0])
        
        outline_points.append(line_points)
        
    print(len(outline_points))
        
    return np.array(outline_points).reshape((no_of_lines,2,3)),lines

In [328]:
points = open("Datasets/Original Foot Data/Processed/sb1255ll.pts")
point_array = process_data(points)

In [329]:
outline =get_outline(project_points(point_array))
outline_points= outline[0]
lines=outline[1]

200


In [330]:
plot_graph(point_array,outline_points=outline_points)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: cannot reshape array of size 1200 into shape (100,3)

In [153]:
a

array([[-42.094639,   0.      , 156.270004],
       [ 34.385372,   0.      , 156.270004],
       [-44.994091,   0.      , 196.280014],
       [ 45.159794,   0.      , 196.280014],
       [-40.907536,   0.      , 236.220001],
       [ 44.268547,   0.      , 236.220001],
       [-39.90551 ,   0.      , 277.180023],
       [ 34.580723,   0.      , 277.180023],
       [-31.841209,   0.      , 317.180023],
       [ 31.206987,   0.      , 317.180023]])

In [154]:
a[:,0]

array([-42.094639,  34.385372, -44.994091,  45.159794, -40.907536,
        44.268547, -39.90551 ,  34.580723, -31.841209,  31.206987])